In [1]:
import sympy
from sympy.abc import x,y,z
from cmath import pi
from kinematicBuilder import *

# Two-Link Planar 

## Examples

In [2]:
twp = TwoLinkPlanar(2,3)
twp.directKinSymbols()

Matrix([
[2*cos(q_1) + 3*cos(q_1 + q_2)],
[2*sin(q_1) + 3*sin(q_1 + q_2)],
[                            0],
[                    q_1 + q_2],
[                            0],
[                            0]])

In [3]:
twp.directLambda(pi/2,pi/2)

array([[-1.0000000e+00, -1.2246468e-16,  0.0000000e+00, -3.0000000e+00],
       [ 1.2246468e-16, -1.0000000e+00,  0.0000000e+00,  2.0000000e+00],
       [ 0.0000000e+00,  0.0000000e+00,  1.0000000e+00,  0.0000000e+00],
       [ 0.0000000e+00,  0.0000000e+00,  0.0000000e+00,  1.0000000e+00]])

In [4]:
twp.inverseLambda(-1,2)

array([[0.46364761],
       [2.30052398]])

# Two-Link Planar on rotatory base

## Examples

In [5]:
tlb = TwoLinkAndBase(50,50)
tlb.directKinHomoTSymbols()

Matrix([
[cos(q_1)*cos(q_2 + q_3), -sin(q_1), -sin(q_2 + q_3)*cos(q_1), (50*cos(q_2) + 50*cos(q_2 + q_3))*cos(q_1)],
[sin(q_1)*cos(q_2 + q_3),  cos(q_1), -sin(q_1)*sin(q_2 + q_3), (50*cos(q_2) + 50*cos(q_2 + q_3))*sin(q_1)],
[         sin(q_2 + q_3),         0,           cos(q_2 + q_3),            50*sin(q_2) + 50*sin(q_2 + q_3)],
[                      0,         0,                        0,                                          1]])

In [6]:
tlb.inverseKinSymbols()

Matrix([
[                                                                                                                 atan2(y, x)],
[atan2(z, sqrt(x**2 + y**2)) - atan2(50*sqrt(1 - (x**2/5000 + y**2/5000 + z**2/5000 - 1)**2), x**2/100 + y**2/100 + z**2/100)],
[                                                                                 acos(x**2/5000 + y**2/5000 + z**2/5000 - 1)]])

In [7]:
tlb.directLambda(0,pi/4,pi/4)

array([[ 6.12323400e-17, -0.00000000e+00, -1.00000000e+00,
         3.53553391e+01],
       [ 0.00000000e+00,  1.00000000e+00, -0.00000000e+00,
         0.00000000e+00],
       [ 1.00000000e+00,  0.00000000e+00,  6.12323400e-17,
         8.53553391e+01],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         1.00000000e+00]])

In [8]:
tlb.inverseLambda(4,0,4)

array([[ 0.        ],
       [-0.72879941],
       [ 3.02839514]])

# 3DOF Spherical wrist 

## Examples

In [9]:
rw = SphericalWrist("xyx")

Singularities in {beta: 0}


In [10]:
rw.directKinHomoTSymbols()

Matrix([
[          cos(q_2),                              -sin(q_2)*sin(q_3),                              -sin(q_2)*cos(q_3), 0],
[-sin(q_1)*sin(q_2), -sin(q_1)*sin(q_3)*cos(q_2) + cos(q_1)*cos(q_3), -sin(q_1)*cos(q_2)*cos(q_3) - sin(q_3)*cos(q_1), 0],
[ sin(q_2)*cos(q_1),  sin(q_1)*cos(q_3) + sin(q_3)*cos(q_1)*cos(q_2), -sin(q_1)*sin(q_3) + cos(q_1)*cos(q_2)*cos(q_3), 0],
[                 0,                                               0,                                               0, 1]])

In [11]:
rw.inverseKinSymbols()

Matrix([
[  atan2(sin(beta)*sin(gamma), sin(beta)*cos(gamma))],
[                                    acos(cos(beta))],
[atan2(-sin(alpha)*sin(beta), -sin(beta)*cos(alpha))]])

In [12]:
rw.inverse(0,0,0)

array([[ 0.        ],
       [ 0.        ],
       [-3.14159265]])

# Decoupled kinematics 6DOF

## Examples

In [13]:
d6d = Decoupled6DOF(2,3)
d6d.directKinHomoTSymbols()

Singularities in {beta: 0}
Singularities in {beta: 0}
Singularities in {beta: 0}


Matrix([
[                                                                        -sin(q_1)*sin(q_5)*sin(q_6)*cos(q_2 + q_3) - sin(q_5)*sin(q_2 + q_3)*cos(q_6) + cos(q_1)*cos(q_5)*cos(q_2 + q_3),                                         -sin(q_1)*cos(q_5) - sin(q_5)*sin(q_6)*cos(q_1),                                                                           sin(q_1)*sin(q_5)*sin(q_6)*sin(q_2 + q_3) - sin(q_5)*cos(q_6)*cos(q_2 + q_3) - sin(q_2 + q_3)*cos(q_1)*cos(q_5),                                                                         -(2*sin(q_2) + 3*sin(q_2 + q_3))*sin(q_5)*cos(q_6) - (2*cos(q_2) + 3*cos(q_2 + q_3))*sin(q_1)*sin(q_5)*sin(q_6) + (2*cos(q_2) + 3*cos(q_2 + q_3))*cos(q_1)*cos(q_5)],
[(-sin(q_4)*sin(q_6)*cos(q_5) + cos(q_4)*cos(q_6))*sin(q_1)*cos(q_2 + q_3) + (-sin(q_4)*cos(q_5)*cos(q_6) - sin(q_6)*cos(q_4))*sin(q_2 + q_3) - sin(q_4)*sin(q_5)*cos(q_1)*cos(q_2 + q_3), (-sin(q_4)*sin(q_6)*cos(q_5) + cos(q_4)*cos(q_6))*cos(q_1) + sin(q_1)*sin(q_4)*sin(q_5), -(-sin(q_4)*si

In [14]:
d6d.inverseKinSymbols()

Matrix([
[                                                                                                                                                                             atan2(y, x)],
[                                                               atan2(z, sqrt(x**2 + y**2)) - atan2(3*sqrt(1 - (x**2/12 + y**2/12 + z**2/12 - 13/12)**2), x**2/4 + y**2/4 + z**2/4 - 5/4)],
[                                                                                                                                               acos(x**2/12 + y**2/12 + z**2/12 - 13/12)],
[                                                                                                                        -atan2(y, x) + atan2(sin(beta)*sin(gamma), sin(beta)*cos(gamma))],
[-acos(x**2/12 + y**2/12 + z**2/12 - 13/12) + acos(cos(beta)) - atan2(z, sqrt(x**2 + y**2)) + atan2(3*sqrt(1 - (x**2/12 + y**2/12 + z**2/12 - 13/12)**2), x**2/4 + y**2/4 + z**2/4 - 5/4)],
[                                                  

In [15]:
d6d.inverse(1.5,1.5,0,0,pi/2,0)

array([[ 0.78539816],
       [-1.62975605],
       [ 2.3579306 ],
       [-0.78539816],
       [ 0.84262177],
       [-3.14159265]])